In [1]:
import findspark

findspark.init()

In [60]:
import pandas as pd
import pyspark
import pyspark.pandas as ps
from datasets import load_dataset
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType
from pyspark.sql.window import Window

In [3]:
spark = SparkSession.builder \
    .appName("semantic-memorization") \
    .master("local[*]") \
    .config("spark.driver.memory", "32g") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/08/31 00:43:58 WARN Utils: Your hostname, alvin resolves to a loopback address: 127.0.1.1; using 172.18.134.157 instead (on interface eth0)
23/08/31 00:43:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/31 00:43:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
schema = "deduped"
model_size = "70m"
split_name = f"{schema}.{model_size}"
dataset = load_dataset("EleutherAI/pythia-memorized-evals")[split_name].to_pandas().rename(columns={"index": "sequence_id"})
dataset.tokens = dataset.tokens.map(lambda x: x.tolist())

Found cached dataset parquet (/home/alvin/.cache/huggingface/datasets/EleutherAI___parquet/EleutherAI--pythia-memorized-evals-623aaa371a33821a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/16 [00:00<?, ?it/s]

In [37]:
columns = ['sequence_id', 'tokens']
main_df = ps.from_pandas(dataset[columns]).to_spark().alias('main')

/home/alvin/lib/miniconda/lib/python3.10/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [6]:
# (sequence_id, frequency)
sequence_duplicates = (
    load_dataset(f"usvsnsp/{schema}-num-duplicates")["train"].to_pandas().rename(columns={"Index": "sequence_id", "Counts": "frequency"})
)
# (token_id, frequency)
memorized_frequencies = (
    load_dataset(f"usvsnsp/{schema}-num-frequencies")["memorized"].to_pandas().rename(columns={"TokenID": "token_id", "Frequency": "frequency"})
)
# (token_id, frequency)
non_memorized_frequencies = (
    load_dataset(f"usvsnsp/{schema}-num-frequencies")["non_memorized"]
    .to_pandas()
    .rename(columns={"TokenID": "token_id", "Frequency": "frequency"})
)

Found cached dataset parquet (/home/alvin/.cache/huggingface/datasets/usvsnsp___parquet/usvsnsp--deduped-num-duplicates-0fa46dc2ec358c44/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset parquet (/home/alvin/.cache/huggingface/datasets/usvsnsp___parquet/usvsnsp--deduped-num-frequencies-fe78b5f530de7c72/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset parquet (/home/alvin/.cache/huggingface/datasets/usvsnsp___parquet/usvsnsp--deduped-num-frequencies-fe78b5f530de7c72/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [38]:
# sequence_duplicates_df = ps.from_pandas(sequence_duplicates).to_spark()
memorized_frequencies_df = ps.from_pandas(memorized_frequencies).to_spark().alias('memorized')
non_memorized_frequencies_df = ps.from_pandas(non_memorized_frequencies).to_spark().alias('non_memorized')

/home/alvin/lib/miniconda/lib/python3.10/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
/home/alvin/lib/miniconda/lib/python3.10/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [8]:
memorized_frequencies_df.count()

60000

In [9]:
non_memorized_frequencies_df.count()

60000

In [13]:
memorized_frequencies_df.show(5)

+--------+---------+
|token_id|frequency|
+--------+---------+
|       0|        6|
|       1|        0|
|       2|  2104276|
|       3| 26657600|
|       4|  7175770|
+--------+---------+
only showing top 5 rows



In [11]:
non_memorized_frequencies_df.show(5)

+--------+---------+
|token_id|frequency|
+--------+---------+
|       0|      160|
|       1|        0|
|       2|155018971|
|       3|663093972|
|       4|122914143|
+--------+---------+
only showing top 5 rows



In [12]:
main_df.show(5)

+-----------+--------------------+
|sequence_id|              tokens|
+-----------+--------------------+
|        441|[5584, 4196, 1228...|
|        447|[50262, 61, 2099,...|
|        792|[475, 50272, 953,...|
|       1539|[424, 380, 16101,...|
|       1705|[3498, 2262, 2369...|
+-----------+--------------------+
only showing top 5 rows



23/08/31 00:46:19 WARN TaskSetManager: Stage 8 contains a task of very large size (2211 KiB). The maximum recommended task size is 1000 KiB.


In [39]:
main_df = main_df.select(
    "sequence_id",
    F.posexplode("tokens").alias("token_index", "token_id"),
)

In [58]:
combined_df = main_df \
    .join(memorized_frequencies_df, on="token_id", how="left") \
    .join(non_memorized_frequencies_df, on="token_id", how="left") \
    .select(
        F.col('sequence_id'),
        F.col('token_index'),
        F.col('token_id'),
        F.col('memorized.frequency').alias('memorized_frequency'),
        F.col('non_memorized.frequency').alias('non_memorized_frequency'),
    )

In [68]:
combined_df.show(5)

23/08/31 01:16:09 WARN TaskSetManager: Stage 55 contains a task of very large size (2210 KiB). The maximum recommended task size is 1000 KiB.


+-----------+-----------+--------+-------------------+-----------------------+
|sequence_id|token_index|token_id|memorized_frequency|non_memorized_frequency|
+-----------+-----------+--------+-------------------+-----------------------+
|        441|          3|     187|          228489488|            11512507473|
|        441|          2|    1228|             988259|               42548437|
|        441|          5|       4|            7175770|              122914143|
|        441|          0|    5584|              38938|                8728403|
|        441|          4|    1036|            1750555|               69787000|
+-----------+-----------+--------+-------------------+-----------------------+
only showing top 5 rows



In [71]:
window = Window.partitionBy('sequence_id').orderBy('token_index')

test_df = combined_df.groupby('sequence_id').agg(
    F.collect_list('token_id')
)

In [75]:
test_df.show(5)

23/08/31 01:17:20 WARN TaskSetManager: Stage 70 contains a task of very large size (2210 KiB). The maximum recommended task size is 1000 KiB.
23/08/31 01:17:20 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-4df93ba6-804f-4900-aa85-ec98ac4e26a1/14/temp_shuffle_17c70ce0-4817-4192-a234-f84c41fbb9a9
23/08/31 01:17:20 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-4df93ba6-804f-4900-aa85-ec98ac4e26a1/3f/temp_shuffle_c0d3a4aa-d583-4d29-907d-d1f1f106b133
23/08/31 01:17:20 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-4df93ba6-804f-4900-aa85-ec98ac4e26a1/0a/temp_shuffle_3c9199b0-de35-4f10-b874-0d7e904236e3
23/08/31 01:17:20 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-4df93ba6-804f-4900-aa85-ec98ac4e26a1/3e/temp_shuffle_a8a260dc-a600-45d2-8f28-ac30bad17431
23/08/31 01:17:20 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-4df93ba6-804f-4900-aa85-ec98ac4e26a1/0c/temp_shuffle_fcf1e4fe-0a6c-4b63-8f72-b8dedcd1f56e
23/08/31 01:17:20 WARN DiskBlockO

Py4JJavaError: An error occurred while calling o610.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 70.0 failed 1 times, most recent failure: Lost task 2.0 in stage 70.0 (TID 879) (172.18.134.157 executor driver): java.io.FileNotFoundException: /tmp/blockmgr-4df93ba6-804f-4900-aa85-ec98ac4e26a1/09/temp_shuffle_34897c66-d083-4b9e-b225-50e79e6d0773 (Too many open files)
	at java.base/java.io.FileOutputStream.open0(Native Method)
	at java.base/java.io.FileOutputStream.open(FileOutputStream.java:298)
	at java.base/java.io.FileOutputStream.<init>(FileOutputStream.java:237)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:140)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:159)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:306)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java

In [ ]:
# .over(window).alias('tokens'),